# 4章

以下で実行するコードには確率的な処理が含まれていることがあり、コードの出力結果と本書に記載されている出力例が異なることがあります。

In [1]:
# 4-2

import torch
from transformers import BertJapaneseTokenizer

## トークナイザー

文章をトークナイザーでトークン化し、モデルに学習させる。<br>
学習済みトークナイザーの読み込みは、`transformers.BertJapaneseTokenizer`クラスの`pretrained()`メソッドを用いる。

In [2]:
# 4-3
# UHT BERTを呼び出す

model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

BERTのトークナイザーは`MeCab`(形態素解析)→`WordPiece`(単語をトークン化)という流れで処理する。<br>
特に指定しない限り、`MeCab`には`ipadic`が辞書として使用される。

`WordPiece`は32000のトークンを持ち、トークンを用いて単語を分割している。<br>
`WordPiece`のトークンは`vocab`インスタンス変数で確認できる。

In [13]:
tokenizer.vocab

OrderedDict([('[PAD]', 0),
             ('[UNK]', 1),
             ('[CLS]', 2),
             ('[SEP]', 3),
             ('[MASK]', 4),
             ('の', 5),
             ('、', 6),
             ('に', 7),
             ('。', 8),
             ('は', 9),
             ('た', 10),
             ('を', 11),
             ('で', 12),
             ('と', 13),
             ('が', 14),
             ('し', 15),
             ('て', 16),
             ('1', 17),
             ('な', 18),
             ('年', 19),
             ('れ', 20),
             ('い', 21),
             ('あ', 22),
             ('(', 23),
             (')', 24),
             ('2', 25),
             ('さ', 26),
             ('こ', 27),
             ('も', 28),
             ('か', 29),
             ('##する', 30),
             ('ある', 31),
             ('日', 32),
             ('いる', 33),
             ('する', 34),
             ('・', 35),
             ('「', 36),
             ('月', 37),
             ('」', 38),
             ('19', 39),
             ('から', 40

<br>

トークン化してみる

In [14]:
# 4-4
# 文章のトークン化

tokenizer.tokenize("明日は自然言語処理の勉強をしよう。")

['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '。']

In [15]:
# 4-5

tokenizer.tokenize("明日はマシンラーニングの勉強をしよう。")

['明日', 'は', 'マシン', '##ラー', '##ニング', 'の', '勉強', 'を', 'しよ', 'う', '。']

`##`はサブワードに分類されたときの最初以外の単語につくもの

In [16]:
# 4-6
tokenizer.tokenize('機械学習を中国語にすると机器学习だ。')

['機械', '学習', 'を', '中国', '語', 'に', 'する', 'と', '机', '器', '学', '[UNK]', 'だ', '。']

WordPieceの語彙に含まれていない場合`[UNK]`となる

実際にモデルに入力するには、トークンをID(ユニークな数値)に変換(符号化)する必要がある。<br>
`encode()`メソッドで符号化する。

In [18]:
# 4-7

input_ids = tokenizer.encode("明日は自然言語処理の勉強をしよう。")
print(input_ids)

[2, 11475, 9, 1757, 1882, 2762, 5, 8192, 11, 2132, 205, 8, 3]


<br>

ID列をトークン列に再変換するには`convert_ids_to_tokens()`メソッドを使用する。

In [10]:
# 4-8
tokenizer.convert_ids_to_tokens(input_ids)

['[CLS]', '明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '。', '[SEP]']

<br>

BERTに文章を入力する際は、各文章の長さを統一する必要がある。<br>
文章の長さを統一するには、特殊トークン`[PAD]`で埋める。これには`tokeniser`インスタンスを関数として呼び出す。<br>

In [24]:
# 4-9

text = "明日の天気は晴れだ"
tokenizer.tokenize(text)

['明日', 'の', '天気', 'は', '晴れ', 'だ']

In [25]:
# 4-9

encoding = tokenizer(text, max_length=12, padding='max_length', truncation=True)
print(encoding)

{'input_ids': [2, 11475, 5, 11385, 9, 16577, 75, 3, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]}


`tokenizer()`関数は辞書として返す。`padding='max_length'`と`truncation=True`が与えられると、`max_length`引数に長さが揃えられる。

`input_ids`: 符号化した文章<br>
`token_type_ids`: 複数文章を比較する際に区別する目的で使用する<br>
`attention_mask`: Attention機構にどこに注目させるかを決める。`[PAD]`特殊トークンは`0`に変換される。

In [27]:
# 4-9

tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print(tokens)

['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


`[PAD]`で埋められていることがわかる。

実験的に`max_length`引数を小さくしてみる。

In [35]:
# 4-10

encoding = tokenizer(text, max_length=6, padding='max_length', truncation=True)
print(f"encoded text: {encoding['input_ids']}")

tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print(f"re-tokenize text: {tokens}")

encoded text: [2, 11475, 5, 11385, 9, 3]
re-tokenize text: ['[CLS]', '明日', 'の', '天気', 'は', '[SEP]']


`max_length`より長い文章は、トークン化の際にカットされる。

`tokenizer()`にリストとして複数の文章を渡せば、同時に符号化することができる。

In [37]:
# 4-11

text_lst = ["明日の天気は晴れだ", "パソコンが急に動かなくなった"]
tokenizer(text_lst, max_length=10, padding='max_length', truncation=True)['input_ids']

[[2, 11475, 5, 11385, 9, 16577, 75, 3, 0, 0],
 [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 3]]

<br>

リスト内で最大長に揃えたい場合、`tokenizer()`関数の`padding`引数に`longest`を渡す。

In [38]:
# 4-13

tokenizer(text_lst, padding='longest')['input_ids']

[[2, 11475, 5, 11385, 9, 16577, 75, 3, 0, 0],
 [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 3]]

<br>

PyTorchに符号化した文章を渡す場合、`torch.tensor`型に変換する必要がある。`tokenizer()`関数の`return_tensors`引数に`pt`を指定する。<br>
この際、二次元ベクトルとして帰ってくることに注意する。

In [43]:
# 4-13

tokenizer(text_lst, padding='longest', return_tensors='pt')['input_ids']

tensor([[    2, 11475,     5, 11385,     9, 16577,    75,     3,     0,     0],
        [    2,  6311,    14,  1132,     7, 16084,   332,    58,    10,     3]])

## BERTモデル

符号化されたデータをBERTに入力し、それぞれのトークンに対するベクトルを出力する。

In [46]:
from transformers import BertModel

model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


モデルをGPUに載せる場合、以下のコードを追加する。

In [ ]:
# model = model.cuda()

<br>

モデルの概要は`BertModel.config`インスタンス変数で出力する。

In [47]:
model.config

BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-whole-word-masking",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.29.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

`num_hidden_layers`: 隠れ層の数<br>
`hidden_size`: 出力次元数<br>
`max_position_embeddings`: 最大入力トークン数

BERTでは、符号化された文章を入力することで、BERT最終層の出力を得ることができる。<br>
まとめて処理する文章の数をバッチサイズと呼ぶ。<br>

In [52]:
# 4-16

text_lst = [
    "明日は自然言語処理の勉強をしよう。", 
    "明日はマシーンラーニングの勉強をしよう。",
]

# 文章を符号化する
encoding = tokenizer(
    text_lst, max_length=32, padding='max_length', truncation=True, return_tensors='pt'
)

# データをGPUに載せる場合
# encoding = {key: value.cuda() for key, value in encoding.items()}

# BERTで処理する
output = model(**encoding) # 入力はdict, valueは2次元のtorch.Tensorを可変長引数**kwargsで渡す
last_hidden_state = output.last_hidden_state # 最終層の出力

In [56]:
last_hidden_state.size()

torch.Size([2, 32, 768])

`2`: バッチサイズ(まとめて処理する文章の数)<br>
`32`: 系列長(符号化した文章の長さ)<br>
`768`: 出力層

`i`番目の文章に含まれる`j`番目のトークンに対する出力は、`last_hidden_state[i][j]`となる。

また、BERTで推論のみを行う場合、`with`ブロックで`torch.no_grad()`を囲むようにする。

In [ ]:
with torch.no_grad():
    output = model(**encoding)
    last_hidden_state = output.last_hidden_state

<br>

GPUメモリ上に変数を置いていたり、`numpy.ndarray`型に変換したりする場合、メインメモリに移す必要がある。

In [ ]:
last_hidden_state.cpu() # メインメモリに移す
last_hidden_state.numpy() # numpy.ndarrayに変換